In [1]:
import mlrun

# mlrun.set_env_from_file("mlrun.env")

project = mlrun.get_or_create_project(name="sense-private-git", context="./")

> 2022-07-13 21:43:13,176 [info] loaded project sense-private-git from MLRun DB


# Code Overview

In [2]:
!cat module.py

def external_function():
    return "hello there"

In [3]:
!cat test.py

from module import external_function

def handler(context):
    context.logger.info(external_function())


# Public Repo

This is creating a new function to run the job

In [4]:
fn = mlrun.new_function(
    name="git-test",
    kind="job",
    image="mlrun/mlrun",
)

This is adding your git repo to the job and specifying to pull it at runtime. The handler is in the format of `<FILE>.<FUNCTION>`. In this case, my `test.py` file has a function called `handler`

In [5]:
fn.with_source_archive(
    source="git://github.com/igz-us-sales/sense-git.git", # <------ this repo is public
    handler="test.handler",
    pull_at_runtime=True
)

This is running the job. It does not require a secret as the repo is public

In [6]:
fn.run(
    local=False,
)

> 2022-07-13 21:43:56,015 [info] starting run git-test-handler uid=4911a126bfd14e0e96e297a8e807309b DB=http://mlrun-api:8080
> 2022-07-13 21:43:56,212 [info] Job is running in the background, pod: git-test-handler-brwt4
> 2022-07-13 21:44:00,026 [info] extracting source from git://github.com/igz-us-sales/sense-git.git to /mlrun/code
> 2022-07-13 21:44:00,066 [info] hello there
> 2022-07-13 21:44:00,102 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sense-private-git,...e807309b,0,Jul 13 21:44:00,completed,git-test-handler,v3io_user=nickkind=jobowner=nickmlrun/client_version=1.0.4host=git-test-handler-brwt4,,,,


> 2022-07-13 21:44:02,508 [info] run executed, status=completed


# Private Repo

### Create Kubernetes Secret

You only need to add this once per project. Do not commit this to git

In [ ]:
secrets = {'GIT_TOKEN': 'XXXXXXXXXXXXXXXXXXXXX'}
project.set_secrets(secrets=secrets, provider="kubernetes")

### Run

This is creating a new function to run the job

In [7]:
fn = mlrun.new_function(
    name="git-test",
    kind="job",
    image="mlrun/mlrun",
)

This is adding your git repo to the job and specifying to pull it at runtime. The handler is in the format of `<FILE>.<FUNCTION>`. In this case, my `test.py` file has a function called `handler`

In [10]:
fn.with_source_archive(
    source="git://github.com/igz-us-sales/sense-git-private.git", # <---- this repo is private
    handler="test.handler",
    pull_at_runtime=True
)

This is running the job and adding your secret. The secret will be pulled at runtime from the Kubernetes cluster

In [11]:
fn.run(
    local=False,
    runspec=mlrun.new_task().with_secrets('kubernetes', ['GIT_TOKEN'])
)

> 2022-07-13 21:45:32,556 [info] starting run git-test-handler uid=d0f2bb82a0a04e9587615ab9fd22ddef DB=http://mlrun-api:8080
> 2022-07-13 21:45:32,737 [info] Job is running in the background, pod: git-test-handler-vjnvk
> 2022-07-13 21:45:36,638 [info] extracting source from git://github.com/igz-us-sales/sense-git-private.git to /mlrun/code
> 2022-07-13 21:45:36,677 [info] hello there
> 2022-07-13 21:45:36,712 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
sense-private-git,...fd22ddef,0,Jul 13 21:45:36,completed,git-test-handler,v3io_user=nickkind=jobowner=nickmlrun/client_version=1.0.4host=git-test-handler-vjnvk,,,,


> 2022-07-13 21:45:38,965 [info] run executed, status=completed
